In [0]:
dbutils.widgets.text("job_id", "", "Analysis Job ID")
dbutils.widgets.text("schema_id", "", "Schema ID")
job_id = dbutils.widgets.get("job_id")
schema_id = dbutils.widgets.get("schema_id")
print(f"Using job_id: {job_id}, schema_id: {schema_id}")

Using job_id: 12, schema_id: 1


In [0]:
%run ./postgres_client_util

In [0]:

upload_directory = execute_query(f"select upload_directory from {job_conf['lakebase_schema_name']}.extraction_jobs where id = {job_id}", fetch_all=True)[0]['upload_directory']

In [0]:
parse_job_output = dbutils.notebook.run("./ai-document-parser", 0, {
  "ai_parse_document_output_table": job_conf['ai_parse_document_output_table'],
  "source_volume_path" : upload_directory
  })

In [0]:
import json
documents_parsed = json.loads(parse_job_output)

In [0]:
documents_info = execute_query(f"""
                               SELECT 
                                * 
                               FROM 
                                {job_conf['lakebase_schema_name']}.documents 
                               WHERE 
                                job_id = {job_id}""", fetch_all=True
                              )

In [0]:
import pandas as pd
documents_info = pd.DataFrame(documents_info)
paths_md5 = pd.DataFrame(documents_parsed)

In [0]:
documents_metadata = documents_info.merge(paths_md5, left_on='file_path', right_on='path', how='inner').drop(columns=['path'])
display(documents_metadata)

id,job_id,filename,file_path,file_size,upload_time,file_content_checksum
34,12,batch1-0001.pdf,/Volumes/fouad_demos/information_extraction/sanabil_documents/job_12/batch1-0001.pdf,258418,2025-09-30T12:15:55.761Z,47d8af4a3aeb33dbb68c0816106e6412
35,12,batch1-0004.pdf,/Volumes/fouad_demos/information_extraction/sanabil_documents/job_12/batch1-0004.pdf,225544,2025-09-30T12:15:58.407Z,6f783bfda93e313f8dc3e4a48d7b78f7
36,12,batch1-0008.pdf,/Volumes/fouad_demos/information_extraction/sanabil_documents/job_12/batch1-0008.pdf,212882,2025-09-30T12:16:01.294Z,671ed1b6f00c23a41b4edad83b298693
37,12,batch1-0011.pdf,/Volumes/fouad_demos/information_extraction/sanabil_documents/job_12/batch1-0011.pdf,179683,2025-09-30T12:16:03.982Z,1d3eb92226db5715ff8cbef06bf02eeb


In [0]:

extraction_schema = execute_query(f"select fields from {job_conf['lakebase_schema_name']}.extraction_schemas where id = {schema_id}", fetch_all=True)[0]['fields']

In [0]:
import json

schema_list = json.loads(extraction_schema)
items = []
items_str = []
for obj in schema_list:
    obj.pop('id', None)
    obj.pop('type', None)
    obj.pop('required', None)
    if 'name' in obj:
        obj['element_to_extract'] = obj.pop('name')
    element = obj.get('element_to_extract', '')
    description = obj.get('description', '')
    items.append(obj)
    items_str.append(f"{element}: {description}")
elements_to_extract = "\n".join(items_str)
elements_to_extract
items

[{'description': 'The number of the invoice',
  'element_to_extract': 'Invoice number'},
 {'description': 'The issuance date', 'element_to_extract': 'date'},
 {'description': '', 'element_to_extract': 'seller address'},
 {'description': '', 'element_to_extract': 'seller tax number'},
 {'description': '', 'element_to_extract': 'client address'},
 {'description': '', 'element_to_extract': 'client tax number'},
 {'description': '', 'element_to_extract': 'vat percentage'},
 {'description': 'the vat amount in usd', 'element_to_extract': 'vat amount'},
 {'description': '', 'element_to_extract': 'net worth'}]

{'db_password_scope': 'fouad_demos',
 'lakebase_db_password_key': 'lakebase_db_password',
 'lakebase_instance_host': 'instance-3d40d832-a0b1-4db5-ae28-b68a1f1be773.database.azuredatabricks.net',
 'lakebase_db_name': 'portfolio_manager',
 'lakebase_schema_name': 'information_extraction',
 'ai_parse_document_output_table': 'fouad_demos.information_extraction.ai_parse_document_output'}

In [0]:
propmpt  = f"""
please only extract the following information ( do not extract any other information ) and return the results as a simple json format without backticks or 'json' keyword : \n {items}  \n the content here : 
"""
propmpt

"\nplease only extract the following information ( do not extract any other information ) and return the results as a simple json format without backticks or 'json' keyword : \n [{'description': 'The number of the invoice', 'element_to_extract': 'Invoice number'}, {'description': 'The issuance date', 'element_to_extract': 'date'}, {'description': '', 'element_to_extract': 'seller address'}, {'description': '', 'element_to_extract': 'seller tax number'}, {'description': '', 'element_to_extract': 'client address'}, {'description': '', 'element_to_extract': 'client tax number'}, {'description': '', 'element_to_extract': 'vat percentage'}, {'description': 'the vat amount in usd', 'element_to_extract': 'vat amount'}, {'description': '', 'element_to_extract': 'net worth'}]  \n the content here : \n"

In [0]:
document_metadata_spark = spark.createDataFrame(documents_metadata)
document_metadata_spark.createOrReplaceTempView('documents_metadata')
display(document_metadata_spark)

id,job_id,filename,file_path,file_size,upload_time,file_content_checksum
34,12,batch1-0001.pdf,/Volumes/fouad_demos/information_extraction/sanabil_documents/job_12/batch1-0001.pdf,258418,2025-09-30T12:15:55.761Z,47d8af4a3aeb33dbb68c0816106e6412
35,12,batch1-0004.pdf,/Volumes/fouad_demos/information_extraction/sanabil_documents/job_12/batch1-0004.pdf,225544,2025-09-30T12:15:58.407Z,6f783bfda93e313f8dc3e4a48d7b78f7
36,12,batch1-0008.pdf,/Volumes/fouad_demos/information_extraction/sanabil_documents/job_12/batch1-0008.pdf,212882,2025-09-30T12:16:01.294Z,671ed1b6f00c23a41b4edad83b298693
37,12,batch1-0011.pdf,/Volumes/fouad_demos/information_extraction/sanabil_documents/job_12/batch1-0011.pdf,179683,2025-09-30T12:16:03.982Z,1d3eb92226db5715ff8cbef06bf02eeb


In [0]:
documents_md5 = ",".join([ f"'{e['file_content_checksum']}'" for e in documents_parsed])
escaped_prompt = json.dumps(propmpt)
inference = spark.sql(f"""
SELECT 
  
  {job_id} as job_id,
  {schema_id} as schema_id,
  m.id as document_id,
  ai_query('databricks-claude-sonnet-4', CONCAT({escaped_prompt}, raw_parsed)) AS extracted_data,
  p.file_content_checksum
FROM 
  {job_conf['ai_parse_document_output_table']} p
JOIN documents_metadata m on p.file_content_checksum=m.file_content_checksum
WHERE
  p.file_content_checksum  in ({documents_md5})
""")
#
# inference_pdf = inference.toPandas()

In [0]:
rows = inference.toPandas().to_dict(orient='records')
display(rows)

document_id,extracted_data,file_content_checksum,job_id,schema_id
35,"{ ""Invoice number"": ""16273983"", ""date"": ""04/01/2017"", ""seller address"": ""Reyes, Holloway and Lee\n38676 Johnson Burg Suite 666\nWest Rebeccamount, SD 02588"", ""seller tax number"": ""909-83-7738"", ""client address"": ""Castillo LLC\n70391 Kelsey Terrace\nGarialand, VT 41740"", ""client tax number"": ""901-88-0463"", ""vat percentage"": ""10%"", ""vat amount"": ""74.46"", ""net worth"": ""744.60"" }",6f783bfda93e313f8dc3e4a48d7b78f7,12,1
34,"{ ""Invoice number"": ""51109338"", ""date"": ""04/13/2013"", ""seller address"": ""Andrews, Kirby and Valdez\n58861 Gonzalez Prairie\nLake Daniellefurt, IN 57228"", ""seller tax number"": ""945-82-2137"", ""client address"": ""Becker Ltd\n8012 Stewart Summit Apt. 455\nNorth Douglas, AZ 95355"", ""client tax number"": ""942-80-0517"", ""vat percentage"": ""10%"", ""vat amount"": ""564.02"", ""net worth"": ""5640.17"" }",47d8af4a3aeb33dbb68c0816106e6412,12,1
37,"{ ""Invoice number"": ""68688408"", ""date"": ""03/20/2016"", ""seller address"": ""Torres and Sons\n4052 Brown Roads\nHaysstad, NJ 06418"", ""seller tax number"": ""953-84-2239"", ""client address"": ""Dennis, Levy and Bowen\n523 Leslie Manors\nLake Colleenville, CT 12053"", ""client tax number"": ""904-82-9186"", ""vat percentage"": ""10%"", ""vat amount"": ""$ 8,30"", ""net worth"": ""$ 82,97"" }",1d3eb92226db5715ff8cbef06bf02eeb,12,1
36,"{ ""Invoice number"": ""74589240"", ""date"": ""12/08/2013"", ""seller address"": ""Hall-Boyd\n8719 Reed Street Apt. 120\nEast Michelleborough, KY 93895"", ""seller tax number"": ""977-76-9492"", ""client address"": ""Haas and Sons\n65628 Brittany Dale\nWalterschester, LA 26239"", ""client tax number"": ""981-73-4585"", ""vat percentage"": ""10%"", ""vat amount"": ""$ 1 179,44"", ""net worth"": ""$ 11 794,44"" }",671ed1b6f00c23a41b4edad83b298693,12,1


In [0]:
table_name = f"{job_conf['lakebase_schema_name']}.extraction_results"
columns = list(rows[0].keys())
columns_str = ', '.join([f'"{col}"' for col in columns])
placeholders = ', '.join([f':{col}' for col in columns])
update_str = ', '.join([f'"{col}"=EXCLUDED."{col}"' for col in columns if col not in ['document_id', 'job_id', 'schema_id']])

upsert_sql = (
    "INSERT INTO {} ({})\nVALUES ({})\nON CONFLICT (document_id, job_id, schema_id)\nDO UPDATE SET {}".format(
        table_name, columns_str, placeholders, update_str
    )
)


In [0]:
for row in rows:
    row_map = dict(row)
    execute_commit_statement(upsert_sql,row_map)